In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

In [3]:
target = pd.DataFrame()

In [4]:
df = pd.read_csv("features_30_sec.csv")

In [5]:
df.drop("filename", axis = 1, inplace=True)

In [6]:
df.head()

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,3805.839606,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,3550.522098,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,3042.260232,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,2184.745799,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,3579.757627,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [7]:
df.shape

(1000, 59)

In [8]:
X = df.drop("label", axis = 1)
y = df["label"]

In [9]:
label_encoder = preprocessing.LabelEncoder() 
y = label_encoder.fit_transform(y) 

In [10]:
len(y)

1000

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify = y)

# Gradient boost

In [12]:
from xgboost import XGBClassifier

In [13]:
model  = XGBClassifier(learning_rate = 0.01)

In [14]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [15]:
y_pred =  model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.71
Confusion Matrix:
 [[18  0  2  0  2  2  1  0  4  1]
 [ 0 28  0  0  0  2  0  0  0  0]
 [ 3  0 20  1  0  4  0  0  1  1]
 [ 0  0  1 17  5  0  1  1  1  4]
 [ 0  0  1  2 22  0  1  0  1  3]
 [ 1  3  0  0  0 23  0  0  0  3]
 [ 0  0  0  2  0  1 25  0  1  1]
 [ 0  0  0  1  1  0  0 27  1  0]
 [ 0  0  1  2  0  0  0  6 20  1]
 [ 5  0  5  2  1  3  0  0  1 13]]


In [16]:
target.loc["xgboost_nest_10", "accuracy"] = accuracy

# Ada boost

In [17]:
from sklearn.ensemble import AdaBoostClassifier

In [18]:
ada_model = AdaBoostClassifier(learning_rate=0.01, random_state=42)

In [19]:
ada_model.fit(X_train, y_train)

AdaBoostClassifier(learning_rate=0.01, random_state=42)

In [20]:
y_pred = ada_model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.32


In [21]:
target.loc["ada_boost_nest_10", "accuracy"] = accuracy_score(y_test, y_pred)

In [22]:
ada_model = AdaBoostClassifier(learning_rate=0.01, random_state=42, n_estimators=1000)
ada_model.fit(X_train, y_train)
y_pred = ada_model.predict(X_test)
print(accuracy_score(y_test, y_pred))
target.loc["ada_boost_nest_1000", "accuracy"] = accuracy_score(y_test, y_pred)

0.31666666666666665


# Scaling

In [23]:
scaler = preprocessing.StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [24]:
model = XGBClassifier(n_estimators=1000)
model.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [25]:
y_pred = model.predict(X_test_scaled)
accuracy_score(y_pred, y_test)

0.78

In [26]:
target.loc["xgboost_nest_1000_scaled", "accuracy"] = accuracy_score(y_test, y_pred)

In [27]:
ada_model.fit(X_train_scaled, y_train)

AdaBoostClassifier(learning_rate=0.01, n_estimators=1000, random_state=42)

In [28]:
y_pred = ada_model.predict(X_test)
accuracy_score(y_pred, y_test)

0.11333333333333333

In [29]:
target.loc["ada_boost_nest_1000_scaled", "accuracy"] = accuracy_score(y_test, y_pred)

# ANN

In [30]:
from tensorflow import keras
from keras.models import Sequential
model_cnn = keras.models.Sequential([
    keras.layers.Dense(2048, activation="relu", input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(1024,activation="relu"),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(512,activation="relu"),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(256,activation="relu"),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(128,activation="relu"),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(64,activation="relu"),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(10, activation="softmax"),
    
])

2023-10-23 09:13:15.659855: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-23 09:13:17.690734: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-23 09:13:17.690781: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-23 09:13:17.694529: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-23 09:13:18.509883: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-23 09:13:18.510840: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [31]:
model_cnn.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics='accuracy')
model_cnn.fit(X_train_scaled, y_train, validation_split=0.2, epochs=100, batch_size=32, verbose = 0)

In [32]:
test_loss, test_accuracy = model_cnn.evaluate(X_test_scaled, y_test, batch_size=128)
print("The test loss is :",test_loss)
print("\nThe test Accuracy is :",test_accuracy*100)

3/3 [==============================] - 0s 11ms/step - loss: 2.1293 - accuracy: 0.7567
The test loss is : 2.1292734146118164

The test Accuracy is : 75.66666603088379


In [33]:
target.loc["ANN_2048_epoch100", "accuracy"] = test_accuracy

In [34]:
target

,accuracy
xgboost_nest_10,0.710000
ada_boost_nest_10,0.320000
ada_boost_nest_1000,0.316667
xgboost_nest_1000_scaled,0.780000
ada_boost_nest_1000_scaled,0.113333
ANN_2048_epoch100,0.756667
